In [1]:
import pandas as pd
import numpy as np

# Load CSV
df = pd.read_csv("xai_analysis_heatmaps.csv")

# Clean column names for convenience
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

In [5]:
# Pivot to have one column per heatmap type (include augmentation!)
pivot_df = df.pivot_table(index=['image_id', 'label', 'augmentation'],
                          columns='heatmap',
                          values='score',
                          aggfunc='first').reset_index()

# Rename for convenience
pivot_df = pivot_df.rename(columns={
    'Lime': 'lime',
    'GradCAM': 'gradcam',
    'Integrated Gradients': 'ig'
})

# Calculate average across heatmaps
pivot_df['total'] = pivot_df[['lime', 'gradcam', 'ig']].mean(axis=1)

# Average per class for all scores (LIME, GradCAM, IG, total)
overall_avg = pivot_df.groupby('label')[['lime', 'gradcam', 'ig', 'total']].mean().reset_index()
overall_avg.columns = ['label', 'Avg Lime', 'Avg GradCAM', 'Avg IG', 'Avg Total']

# Augmented vs Non-Augmented total comparison per class
aug_totals = pivot_df.groupby(['label', 'augmentation'])['total'].mean().unstack()
aug_totals.columns = ['Non-Aug Total', 'Aug Total']
aug_totals = aug_totals.reset_index()

# Merge them
final_agg = pd.merge(overall_avg, aug_totals, on='label')

# Add final total row
total_row = pd.DataFrame({
    'label': ['Total'],
    'Avg Lime': [pivot_df['lime'].mean()],
    'Avg GradCAM': [pivot_df['gradcam'].mean()],
    'Avg IG': [pivot_df['ig'].mean()],
    'Avg Total': [pivot_df['total'].mean()],
    'Non-Aug Total': [pivot_df[pivot_df['augmentation'] == 'No']['total'].mean()],
    'Aug Total': [pivot_df[pivot_df['augmentation'] == 'Yes']['total'].mean()],
})

# Concatenate
final_agg = pd.concat([final_agg, total_row], ignore_index=True)

# Round for clean display
print(final_agg.round(3))

                      label  Avg Lime  Avg GradCAM  Avg IG  Avg Total  \
0     Bacterial Leaf Blight     0.383        0.817   0.717      0.639   
1     Bacterial Leaf Streak     0.467        0.633   0.792      0.631   
2  Bacterial Panicle Blight     0.267        0.533   0.950      0.583   
3                     Blast     0.505        0.632   0.774      0.637   
4                Brown Spot     0.475        0.556   0.693      0.574   
5                Dead Heart     0.431        0.456   0.912      0.600   
6              Downy Mildew     0.415        0.620   0.640      0.558   
7                     Hispa     0.457        0.700   0.729      0.629   
8                    Tungro     0.328        0.539   0.833      0.567   
9                     Total     0.427        0.597   0.771      0.598   

   Non-Aug Total  Aug Total  
0          0.633      0.644  
1          0.622      0.639  
2          0.600      0.567  
3          0.607      0.663  
4          0.592      0.556  
5          0.625

In [6]:
# Ensure key columns are categorical
categorical_cols = ['study', 'label', 'prediction', 'status', 'augmentation', 'heatmap']
df[categorical_cols] = df[categorical_cols].astype('category')

# Group by augmentation and heatmap, compute mean/median score
aug_stats = df.groupby(['augmentation', 'heatmap'])['score'].agg(['mean', 'median']).reset_index()

# Pivot for easier comparison
pivot_stats = aug_stats.pivot(index='heatmap', columns='augmentation', values='mean')
pivot_stats.columns = ['Average (No Aug)', 'Average (Yes Aug)']
pivot_stats['Difference'] = pivot_stats['Average (Yes Aug)'] - pivot_stats['Average (No Aug)']

# Median version (optional)
pivot_median = aug_stats.pivot(index='heatmap', columns='augmentation', values='median')
pivot_median.columns = ['Median (No Aug)', 'Median (Yes Aug)']
pivot_median['Difference'] = pivot_median['Median (Yes Aug)'] - pivot_median['Median (No Aug)']

# Display both
print("Average Score Comparison:")
print(pivot_stats)
print("\nMedian Score Comparison:")
print(pivot_median)


Average Score Comparison:
                      Average (No Aug)  Average (Yes Aug)  Difference
heatmap                                                              
GradCAM                       0.595161           0.598462    0.003300
Integrated Gradients          0.746774           0.795312    0.048538
Lime                          0.406452           0.446154    0.039702

Median Score Comparison:
                      Median (No Aug)  Median (Yes Aug)  Difference
heatmap                                                            
GradCAM                          0.60               0.6        0.00
Integrated Gradients             0.75               0.8        0.05
Lime                             0.40               0.4        0.00
